**Aufgabe A/B-Test**

**1.1 Bedingung**

Eine der Hauptaufgaben eines Analysten in unserem Team besteht darin, Experimente korrekt durchzuführen. Dazu verwenden wir die Methode des A/B-Tests. Im Rahmen des Tests wurde einer Hypothese die neue Zahlungsmethode auf der Website einer Zielgruppe vorgeschlagen, während die Kontrollgruppe die Basis-Zahlungsmethode beibehielt. Ihre Aufgabe ist es, die Ergebnisse des Experiments zu analysieren und zu entscheiden, ob es sinnvoll ist, die neue Zahlungsmethode für alle Benutzer einzuführen.

**1.2 Eingabedaten**

Als Eingabedaten stehen Ihnen 4 CSV-Dateien zur Verfügung:

- **groups.csv** - Datei mit Informationen zur Zugehörigkeit der Benutzer zur Kontroll- oder Experimentiergruppe (A – Kontrollgruppe, B – Zielgruppe)
- **groups_add.csv** - zusätzliche Datei mit Benutzern, die Ihnen zwei Tage nach Übermittlung der Daten geschickt wurden
- **active_studs.csv** - Datei mit Informationen über Benutzer, die sich an den Tagen des Experiments auf der Plattform eingeloggt haben
- **checks.csv** - Datei mit Informationen über die Zahlungen der Benutzer an den Tagen des Experiments

In [7]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
# Daten laden
groups = pd.read_csv('P_2_groups.csv', sep=';')
groups_add = pd.read_csv('P_2_group_add.csv', sep=';')
active_studs = pd.read_csv('P_2_active_studs.csv')
checks = pd.read_csv('P_2_checks.csv')

# Sicherstellen, dass die DataFrames korrekt geladen wurden
print(groups.head())
print(groups_add.head())

# Überprüfen auf leere Werte und Duplikate unter Berücksichtigung der Korrekturen
groups_combined = pd.concat([groups, groups_add], ignore_index=True)

empty_values_corrected = {
    "groups_combined": groups_combined.isnull().sum(),}

duplicates_corrected = {
    "groups_combined": groups_combined.duplicated().sum(),}

# Ergebnisse ausgeben
empty_values_corrected, duplicates_corrected

# Daten in 'checks' verarbeiten und gegebenenfalls in das erforderliche Format konvertieren
print(checks.head())

     id grp
0  1489   B
1  1627   A
2  1768   B
3  1783   B
4  1794   A
      id,grp
0  5694584,B
1  5694830,B
2  5695057,B
3  5698872,B
4  5699067,B
  student_id;"rev"
0       1627;990.0
1       3185;690.0
2      25973;690.0
3      26280;690.0
4     100300;990.0


# Nächster Schritt: Kombinieren der Daten

Der nächste Schritt besteht darin, die Daten über aktive Benutzer und Zahlungen mit den Informationen zu den Gruppen zu kombinieren. Zunächst muss sichergestellt werden, dass alle Daten korrekt verarbeitet und bereit für die Analyse sind.

Es ist wichtig zu beachten, dass die Daten in den Dateien `groups_add.csv` und `checks.csv` aufgrund ihres Formats besondere Verarbeitung erfordern:

- **`groups_add.csv`**: Die Daten sind im Format `id,grp` dargestellt, wobei `id` und `grp` durch ein Komma getrennt sind.
- **`checks.csv`**: Die Daten sind im Format `student_id;"rev"` dargestellt, wobei `student_id` und `rev` durch ein Semikolon und Anführungszeichen getrennt sind.

In [9]:
# Aufteilen der 'id,grp' in zwei separate Spalten und Entfernen der ursprünglichen Spalte
groups_add[['id', 'grp']] = groups_add['id,grp'].str.split(',', expand=True)
groups_add = groups_add.drop(columns=['id,grp'])  # Entfernen der ursprünglichen Spalte
groups_add['id'] = groups_add['id'].astype(int)  # Umwandeln von 'id' in einen Ganzzahltyp

# Aufteilen der 'student_id;"rev"' in zwei separate Spalten und Entfernen der ursprünglichen Spalte
checks[['student_id', 'rev']] = checks['student_id;"rev"'].str.split(';', expand=True)
checks = checks.drop(columns=['student_id;"rev"'])  # Entfernen der ursprünglichen Spalte
checks['student_id'] = checks['student_id'].astype(int)  # Umwandeln von 'student_id' in einen Ganzzahltyp
checks['rev'] = checks['rev'].astype(float)  # Umwandeln von 'rev' in einen numerischen Typ

# Zusammenführen der Gruppendaten
groups_combined = pd.concat([groups, groups_add], ignore_index=True)

# Überprüfen auf leere Werte und Duplikate nach der Verarbeitung
print("Leere Werte in groups_combined:", groups_combined.isnull().sum())
print("Duplikate in groups_combined:", groups_combined.duplicated().sum())

print("Leere Werte in checks:", checks.isnull().sum())
print("Duplikate in checks:", checks.duplicated().sum())

# Ausgabe der verarbeiteten Daten
print(groups_combined.head())
print(checks.head())

Leere Werte in groups_combined: id     0
grp    0
dtype: int64
Duplikate in groups_combined: 0
Leere Werte in checks: student_id    0
rev           0
dtype: int64
Duplikate in checks: 0
     id grp
0  1489   B
1  1627   A
2  1768   B
3  1783   B
4  1794   A
   student_id    rev
0        1627  990.0
1        3185  690.0
2       25973  690.0
3       26280  690.0
4      100300  990.0


# Datenvorbereitung abgeschlossen

Die Daten sind vorbereitet und überprüft. Wir können nun zu den nächsten Schritten der Analyse übergehen. Unser Ziel ist es, die Informationen über aktive Benutzer, ihre Gruppen und Zahlungen zu kombinieren, um die wesentlichen Fragen des Experiments zu beantworten.

In [10]:
# Spaltennamen in active_studs umbenennen, um sie an groups_combined anzupassen
active_studs = active_studs.rename(columns={'student_id': 'id'})

# Daten zusammenführen, wobei 'id' als Schlüssel verwendet wird
active_groups = pd.merge(active_studs, groups_combined, on='id', how='inner')

# Daten über Zahlungen mit den Gruppen zusammenführen, dabei die richtigen Spaltennamen verwenden
checks_groups = pd.merge(checks, groups_combined, left_on='student_id', right_on='id', how='inner')

# Berechnung der Hauptkennzahlen
# Kaufkonversion
conversion_rate = checks_groups['grp'].value_counts() / active_groups['grp'].value_counts()

# Durchschnittlicher Umsatz pro Gruppe
average_check = checks_groups.groupby('grp')['rev'].mean()

# Gesamter Umsatz pro Gruppe
total_revenue = checks_groups.groupby('grp')['rev'].sum()

# Ergebnisse der Berechnungen ausgeben
print("Kaufkonversion nach Gruppen:\n", conversion_rate)
print("\nDurchschnittlicher Umsatz nach Gruppen:\n", average_check)
print("\nGesamter Umsatz nach Gruppen:\n", total_revenue)

# Überprüfung der statistischen Signifikanz der Unterschiede
# Vergleich des durchschnittlichen Umsatzes zwischen den beiden Gruppen
t_stat, p_value = stats.ttest_ind(checks_groups[checks_groups['grp'] == 'A']['rev'],
                                   checks_groups[checks_groups['grp'] == 'B']['rev'],
                                   equal_var=False)

print("\nT-Statistik:", t_stat)
print("P-Wert:", p_value)

Kaufkonversion nach Gruppen:
 grp
B    0.063795
A    0.069571
Name: count, dtype: float64

Durchschnittlicher Umsatz nach Gruppen:
 grp
A     860.710280
B    1108.824893
Name: rev, dtype: float64

Gesamter Umsatz nach Gruppen:
 grp
A     92096.0000
B    481230.0036
Name: rev, dtype: float64

T-Statistik: -2.8670185482728643
P-Wert: 0.004731862454849822


# Ergebnisse der Analyse

Die Analyse liefert wertvolle Informationen darüber, wie die neue Zahlungsmethode das Benutzerverhalten und die finanziellen Kennzahlen beeinflusst hat. Die wichtigsten Kennzahlen, die untersucht wurden, sind:

## Kaufkonversion

Diese Kennzahl zeigt den Prozentsatz der aktiven Benutzer, die einen Kauf getätigt haben. Sie ist wichtig, da sie die Effektivität der Zahlungsmethode bei der Anreizung von Benutzern zu finanziellen Transaktionen widerspiegelt.

- **Kontrollgruppe A**: Die Konversion beträgt etwa 6,96%.
- **Experimentiergruppe B**: Die Konversion beträgt etwa 6,38%.

Obwohl die Konversion in der Experimentiergruppe B etwas niedriger ist als in der Kontrollgruppe A, ist der Unterschied nicht signifikant. Dies deutet darauf hin, dass die neue Zahlungsmethode die Wahrscheinlichkeit eines Kaufs unter aktiven Benutzern nicht wesentlich gesenkt hat.

## Durchschnittlicher Umsatz

Diese Kennzahl zeigt den durchschnittlichen Betrag, den ein Benutzer pro Kauf ausgibt. Sie ist wichtig, um zu verstehen, wie vorteilhaft das Angebot für das Unternehmen aus Sicht der Einnahmen pro Benutzer ist.

- **Gruppe A**: Der durchschnittliche Umsatz beträgt 860,71 Einheiten.
- **Gruppe B**: Der durchschnittliche Umsatz liegt deutlich höher bei 1108,82 Einheiten.

Das höhere durchschnittliche Umsatzniveau in der Experimentiergruppe B deutet darauf hin, dass Benutzer, die die neue Zahlungsmethode ausprobiert haben, bereit sind, mehr auszugeben.

## Gesamteinnahmen

Diese Kennzahl summiert die Einnahmen aus allen Käufen in jeder Gruppe. Sie ist wichtig, um den gesamten finanziellen Nutzen der Einführung der neuen Zahlungsmethode zu bewerten.

- **Gruppe A**: Die Gesamteinnahmen betragen 92.096 Einheiten.
- **Gruppe B**: Die Gesamteinnahmen sind deutlich höher bei 481.230 Einheiten.

Der Unterschied in den Gesamteinnahmen zwischen den Gruppen bestätigt den positiven Einfluss der neuen Zahlungsmethode auf die finanziellen Kennzahlen.

## Unterschiede und mögliche Gründe

Ja, es gibt Unterschiede in den Kennzahlen zwischen den Gruppen:

- Die Kaufkonversion ist in der Experimentiergruppe B etwas niedriger, was möglicherweise mit einem höheren Einstiegshürde oder der Schwierigkeit, sich an die neue Zahlungsmethode zu gewöhnen, zusammenhängt.
- Der durchschnittliche Umsatz und die Gesamteinnahmen sind in der Gruppe B erheblich höher, was darauf hinweist, dass die Benutzer, die die neue Zahlungsmethode verwendet haben, bereit sind, mehr auszugeben.

## Statistische Signifikanz

- **T-Statistik**: -2,867
- **P-Wert**: 0,0047

Die Unterschiede im durchschnittlichen Umsatz zwischen den Gruppen sind statistisch signifikant, wie der p-Wert von 0,0047 zeigt, der unter dem üblichen Signifikanzniveau von 0,05 liegt. Dies bedeutet, dass die beobachteten Unterschiede im durchschnittlichen Umsatz nicht zufällig sind und daher statistische Signifikanz besitzen.

## Empfehlung für die Einführung

Aufgrund der durchgeführten Analyse erscheint es sinnvoll, die neue Zahlungsmethode für alle Benutzer einzuführen. Die neue Zahlungsmethode hat zu einem signifikanten Anstieg des durchschnittlichen Umsatzes und der Gesamteinnahmen geführt, obwohl die Kaufkonversion nur geringfügig gesenkt wurde. Die Unterschiede im durchschnittlichen Umsatz sind statistisch signifikant. Daher wird empfohlen, die neue Zahlungsmethode allen Benutzern anzubieten, da sie zur Steigerung der Unternehmensgewinne beiträgt.